<a href="https://colab.research.google.com/github/fenwickslab/fenwicks/blob/master/tutorials/tutorial4_dvc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -qq -U cufflinks

In [0]:
import tensorflow as tf
import os

In [3]:
if tf.io.gfile.exists('./fenwicks'):
  tf.io.gfile.rmtree('./fenwicks')
!git clone -q https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

In [4]:
ROOT_DIR = 'gs://gs_colab'
PROJECT = 'tutorial4'
MODEL = "InceptionResNetV2" #@param ["InceptionResNetV2", "ResNet50", "ResNet50V2", "InceptionV3", "MobileNetV2", "Xception"]

BATCH_SIZE = 128 #@param ["128", "256", "512"] {type:"raw"}
EPOCHS = 1 #@param {type:"slider", min:0, max:50, step:1}
LEARNING_RATE = 0.001 #@param ["0.001", "0.01"] {type:"raw"}
WARMUP = 0.05 #@param {type:"slider", min:0, max:0.5, step:0.05}


In [5]:
fw.colab_utils.setup_gcs()

In [6]:
data_dir, work_dir = fw.io.get_project_dirs(ROOT_DIR, PROJECT)

In [7]:
base_model = fw.keras_models.get_model(MODEL, ROOT_DIR)

In [8]:
data_dir_local = fw.datasets.untar_data(fw.datasets.URLs.DVC, os.path.join('.', PROJECT))
train_dir_local = os.path.join(data_dir_local, 'dogscats/train')
valid_dir_local = os.path.join(data_dir_local, 'dogscats/valid')

In [9]:
fw.image.check_rgb(train_dir_local)
fw.image.check_rgb(valid_dir_local)

In [10]:
train_fn = os.path.join(data_dir, 'train.tfrec')
valid_fn = os.path.join(data_dir, 'valid.tfrec')
paths_train, y_train, labels = fw.data.data_dir_tfrecord(train_dir_local, train_fn)
paths_valid, y_valid, _ = fw.data.data_dir_tfrecord(valid_dir_local, valid_fn)
n_train, n_valid = len(y_train), len(y_valid)
n_classes = len(labels)

INFO:tensorflow:Output file already exists. Skipping.
INFO:tensorflow:Output file already exists. Skipping.


In [11]:
fw.plt.plot_counts_pie(y_train, labels)

In [12]:
fw.anim.show_image_files(paths_train)

In [13]:
def build_nn(c=256):
  base = base_model.model_func()
  base.trainable = False  
  model = fw.Sequential()
  model.add(base)
  model.add(tf.keras.layers.Flatten())
  model.add(fw.layers.DenseBN(c))
  model.add(fw.layers.Classifier(n_classes))
  return model

In [14]:
steps_per_epoch = n_train // BATCH_SIZE
total_steps = steps_per_epoch * EPOCHS
warmup_steps = int(total_steps * WARMUP)

In [15]:
cosine_decay = tf.train.cosine_decay_restarts
lr_func = fw.train.one_cycle_lr(LEARNING_RATE, total_steps, warmup_steps, cosine_decay)
fw.plt.plot_lr_func(lr_func, total_steps)

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
opt_func = fw.train.adam_optimizer(lr_func)
model_func = fw.train.get_clf_model_func(build_nn, opt_func)

In [17]:
def get_input_func(fn, training):
  h = w = base_model.img_size
  tfms = fw.transform.get_inception_transforms(h, w, training=training, normalizer=base_model.normalizer)
  parser = fw.data.get_tfexample_image_parser(tfms)
  return lambda params: fw.data.tfrecord_ds(fn, parser, params['batch_size'], training=training)

train_input_func = get_input_func(train_fn, True)
valid_input_func = get_input_func(valid_fn, False)

In [18]:
est = fw.train.get_tpu_estimator(steps_per_epoch,  model_func, work_dir, 
  base_model.weight_dir, base_model.weight_vars, trn_bs=BATCH_SIZE, val_bs=n_valid)

est.train(train_input_func, steps=total_steps)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/tutorial4/2019-05-22-19:30:20', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.59.171.130:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0455af6208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.59.171.130:8470', '_evaluation_master': 'grpc://10.59.171.130:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=179, num_shards=

In [19]:
result = est.evaluate(valid_input_func, steps=1)

INFO:tensorflow:Calling model_fn.
ERROR:tensorflow:Operation of type Placeholder (input_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-22T19:33:38Z
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gs://gs_colab/work/tutorial4/2019-05-22-19:30:20/model.ckpt-179
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 7 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.

In [20]:
print(f'Test results: accuracy={result["accuracy"] * 100: .2f}%, loss={result["loss"]: .2f}.')

Test results: accuracy= 99.50%, loss= 0.03.


In [21]:
fw.io.create_clean_dir(work_dir)